# Tutorial: Natural Language Inference

In this short tutorial, we show how to use *ferret* to use and evaluate different gradient based approaches in the task of Natural Language Inference.

We will use`distilbert-base-uncased-finetuned-sst-2-english` as model checkpoint.

In [ ]:
%%capture
!pip install -U ferret-xai

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from ferret import (
    Benchmark,
    GradientExplainer,
    IntegratedGradientExplainer,
    LIMEExplainer,
    SHAPExplainer,
)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
premise = "I first thought that I liked the movie, but upon second thought it was actually disappointing."
hypothesis = "The movie was good."
sample = (premise, hypothesis)

In [ ]:
model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

In [ ]:
ig = IntegratedGradientExplainer(model, tokenizer, multiply_by_inputs=True)
g = GradientExplainer(model, tokenizer, multiply_by_inputs=True)
l = LIMEExplainer(model, tokenizer)

In [ ]:
bench = Benchmark(model, tokenizer, explainers=[ig, g, l])

In [ ]:
# get the prediction from our model
bench.score(sample)

In [ ]:
# explain the contradiction class
exp = bench.explain(sample, target="contradiction")

In [ ]:
# show explanations
bench.show_table(exp)

In [ ]:
# evaluate explanations and show faithfulness metrics
bench.show_evaluation_table(bench.evaluate_explanations(exp, target="contradiction"))